Importar librerias

In [1]:
# Web scraping
#from langchain_community.document_loaders import WebBaseLoader
#from bs4 import BeautifulSoup

import requests
#Embeddings
from langchain_ollama import OllamaEmbeddings
# Prompts
from langchain_core.prompts import ChatPromptTemplate
# doc loaders
from langchain_community.document_loaders import DirectoryLoader, PyPDFLoader, TextLoader, Docx2txtLoader
# ChromaDB
from langchain_community.vectorstores import Chroma
# text splitter
from langchain_text_splitters import RecursiveCharacterTextSplitter
# runnable
from langchain_core.runnables import RunnablePassthrough
# Tools
from langchain.tools import tool
# LLM ollama
from langchain_ollama import ChatOllama
from langchain_core.prompts import ChatPromptTemplate, MessagesPlaceholder
from langchain.agents import create_agent
from langchain_core.messages import HumanMessage, AIMessage, SystemMessage, ToolMessage

# Preparar documentos

## Cargar docs

In [2]:
# Cargar todos los PDFs de la carpeta "Contratos" con PyPDFLoader
loader = DirectoryLoader(
    path="./Contratos",
    glob="**/*.pdf",        # Cargar solo archivos PDF
    loader_cls=PyPDFLoader  # Cargar solo archivos PDF
)

documentos = loader.load()

## Chunking

In [3]:
splitter = RecursiveCharacterTextSplitter(
    chunk_size=500,       # caracteres por chunk
    chunk_overlap=50      # solapamiento entre chunks (mantiene contexto)
)

chunks = splitter.split_documents(documentos)

# Embeddings

In [4]:
embeddings = OllamaEmbeddings(model="llama3.2")

# Chroma

## Crear/Cargar base de datos vectorial

In [5]:
Crear = False
if Crear: #Crear la base de datos (solo la primera vez)
    vectorstore = Chroma.from_documents(
        documents=chunks,
        embedding=embeddings,
        persist_directory="./chroma_db"
    )
else:     # Cargar la base de datos ya creada
    vectorstore = Chroma(
        persist_directory="./chroma_db",
        embedding_function=embeddings
    )

C:\Users\Ame Contreras\AppData\Local\Temp\ipykernel_23204\337347211.py:9: LangChainDeprecationWarning: The class `Chroma` was deprecated in LangChain 0.2.9 and will be removed in 1.0. An updated version of the class exists in the `langchain-chroma package and should be used instead. To use it run `pip install -U `langchain-chroma` and import as `from `langchain_chroma import Chroma``.
  vectorstore = Chroma(


### Agregar nuevos documentos

In [67]:
#vectorstore.add_documents(nuevos_chunks)

# RAG

In [6]:
retriever = vectorstore.as_retriever(search_kwargs={"k": 5})

# LLM

## Agente

###  LLM compartido 


In [7]:
llm = ChatOllama(model="llama3.2", temperature=0)

###  Chains internas + tools

In [8]:
def format_docs(docs):
    return "\n\n".join(d.page_content for d in docs)

#### Analysis

In [9]:
tem_chat_analysis = """ 
Eres un asistente especializado en contratos de arrendamiento. 
Tu función es responder preguntas, proporcionar resúmenes y realizar análisis generales sobre los contratos de arrendamiento.
Utiliza la información disponible en los documentos para ofrecer respuestas precisas y detalladas a las consultas de los usuarios.
"""

chain_analysis = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_chat_analysis + """

Contexto de los documentos:
{context}

Pregunta o solicitud: {question}
""")
    | llm
)

@tool
def chat_analysis(query: str) -> str:
    """Analiza, resume o responde preguntas generales sobre contratos de arrendamiento."""
    result = chain_analysis.invoke(query)
    return result.content

#### Filter

In [10]:
tem_filter_search  = """
Eres un asistente especializado en buscar y filtrar contratos de arrendamiento.
Tu función es ayudar a los usuarios a encontrar contratos específicos basados en criterios como fecha, tipo de contrato, partes involucradas o ubicación.
"""

chain_filter = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_filter_search + """

Contratos encontrados:
{context}

Búsqueda solicitada: {question}
""")
    | llm
)

@tool
def filter_search(query: str, tipo: str = None, año: int = None) -> str:
    """Busca contratos específicos por filtros como tipo, año o arrendador."""
    result = chain_filter.invoke(query)
    return result.content

#### Generation

In [11]:
tem_generation_tool = """
Eres un asistente especializado en crear o sugerir cláusulas, generar borradores o redactar términos de contratos de arrendamiento.
Tu función es ayudar a los usuarios a generar contenido relacionado con contratos de arrendamiento, ya sea creando nuevas cláusulas, sugiriendo redacciones o generando borradores completos.
"""

chain_generation = (
    {"context": retriever | format_docs, "question": RunnablePassthrough()}
    | ChatPromptTemplate.from_template(tem_generation_tool + """

Cláusulas de referencia encontradas:
{context}

Solicitud de generación: {question}

AVISO: Las sugerencias generadas no constituyen asesoría jurídica.
""")
    | llm
)

@tool
def generation_tool(tipo_clausula: str, condiciones: str) -> str:
    """Genera o sugiere cláusulas para contratos de arrendamiento."""
    query = f"{tipo_clausula}: {condiciones}"
    result = chain_generation.invoke(query)
    return result.content

#### Tools

In [12]:
tools = [chat_analysis, filter_search, generation_tool]

### Router prompt

In [13]:
tem_PagPrincipal = """
Eres un asistente especializado en contratos de arrendamiento. 
Tienes acceso a estas herramientas: 
    chat_analysis:      para consultas, resúmenes y análisis generales; 
    filter_search:      cuando el usuario quiere buscar contratos por criterios específicos como fecha, tipo, partes o ubicación; 
    generation_tool:    cuando el usuario quiere crear o sugerir cláusulas, generar borradores o redactar términos. 
Determina cuál usar según la solicitud y extrae los parámetros relevantes.
"""

prompt_agente = ChatPromptTemplate.from_messages([
    ("system", tem_PagPrincipal),
    MessagesPlaceholder("chat_history"),
    ("human", "{input}"),
    MessagesPlaceholder("agent_scratchpad"),
])

agent = create_agent(
    model=llm,
    tools=tools,
    system_prompt=tem_PagPrincipal,
    debug=True  # equivalente a verbose=True
)
#executor = AgentExecutor(agent=agent, tools=tools, verbose=True
#,return_intermediate_steps=True) # para ver qué tools usó

## Manejo de entradas y salidas

In [ ]:
# ── Manejo de historial ──────────────────────────────────────────────────────
chat_history = []

def chat(user_input: str):
    response = agent.invoke({
        "messages": [
            *chat_history,
            {"role": "user", "content": user_input}
        ]
    })

    all_messages = response["messages"]
    final_output = all_messages[-1].content

    # Actualizar historial (solo human + AI final, sin tool messages)
    chat_history.append(HumanMessage(content=user_input))
    chat_history.append(AIMessage(content=final_output))

    # Extraer intermediate steps (tool calls)
    intermediate_steps = []
    for msg in all_messages:
        if isinstance(msg, AIMessage) and msg.tool_calls:
            for tool_call in msg.tool_calls:
                # Buscar el ToolMessage correspondiente
                tool_result = next(
                    (m.content for m in all_messages if isinstance(m, ToolMessage) 
                    and m.tool_call_id == tool_call["id"]),
                    None
                )
                intermediate_steps.append({
                    "tool": tool_call["name"],
                    "params": tool_call["args"],
                    "result": tool_result
                })

    return {"output": final_output, "intermediate_steps": intermediate_steps}

# Pruebas